# Modeling with DecisionTreeClassifier

In [2]:
import pandas as pd
import numpy as np

#Sklearn for model fitting and scroing
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

In [3]:
#Load data
df = pd.read_csv("../Data/training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
69,1,0.661,0.967,0.760,0.183,0.696,0.248,0.003,0.445,0.184,...,0.489,0.815,0.085,0.969,0.920,0.199,0.159,0.323,0.382,0.491
115,0,0.830,0.321,0.110,0.893,0.282,0.338,0.193,0.495,0.173,...,0.339,0.167,0.326,0.359,0.414,0.728,0.539,0.281,0.220,0.848
45,1,0.527,0.432,0.164,0.568,0.747,0.474,0.187,0.303,0.381,...,0.566,0.369,0.069,0.086,0.370,0.446,0.623,0.310,0.284,0.195
81,1,0.028,0.706,0.018,0.984,0.564,0.831,0.697,0.032,0.414,...,0.085,0.511,0.551,0.914,0.061,0.850,0.977,0.557,0.624,0.935
11,0,0.716,0.534,0.857,0.493,0.963,0.852,0.975,0.176,0.048,...,0.348,0.794,0.487,0.749,0.630,0.919,0.448,0.112,0.050,0.385


### Starting

The first stage will be to look at the data in an initial exploratory data analysis. We know that our data is made up of numerical variables and is in a standardized form. The records are complete and there are non null values, so the data is in a clean state.

In [3]:
import ml_utils
df_eda = ml_utils.edaDF(df, "target")
num_var = list(df.columns)
df_eda.setNum(num_var)

In [4]:
df_eda.fullEDA()


In [6]:
df.shape

(250, 201)

### Model fitting

As mentioned above, our data is in a clean state and as such, we will proceed to model fitting. 
The model that is used is the DecisionTreeClassifier.
Conidering that the data set is small-sized (250 rows of data), we will use a 70/30 percent split for the train/test data.

In [15]:
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))

#train size = 70%
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=.7)


Model 1: Here, the DecisionTreeClassifier is used with default hyperparameters 

In [16]:
pipeline_model1 = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0))
    ]

model1 = Pipeline(pipeline_model1)
model1 = model1.fit(X_train, y_train)
print("Training Accuracy:", model1.score(X_train, y_train))
print("Testing Accuracy:", model1.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.6266666666666667


Model 2: The DecisionTreeClassifier is used with the criterion set to "entropy". 
We noticed that there is not so much difference in the testing accuracy 

In [17]:

pipeline_model2 = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0, criterion="entropy"))
    ]
model2 = Pipeline(pipeline_model2)
model2 = model2.fit(X_train, y_train)
print("Training Accuracy:", model2.score(X_train, y_train))
print("Testing Accuracy:", model2.score(X_test, y_test))

Training Accuracy: 1.0
Testing Accuracy: 0.6133333333333333


Model 3: The DecisionTreeClassifier is used with the criterion set default and max_depth set to 5. 
There was improvement in the testing accuracy while the traininig accuracy dropped slightly. This was considered as the best model because further tuning seems not to yield better results.

In [19]:
#Build pipeline
pipeline_best = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(random_state=0, max_depth=5))
    ]

best = Pipeline(pipeline_best)
best = best.fit(X_train, y_train)
print("Training Accuracy:", best.score(X_train, y_train))
print("Testing Accuracy:", best.score(X_test, y_test))

Training Accuracy: 0.9257142857142857
Testing Accuracy: 0.64


### Finishing

In [21]:
print(best.score(X_test, y_test))
print(best)

0.64
Pipeline(steps=[('scaler', StandardScaler()),
                ('DT', DecisionTreeClassifier(max_depth=5, random_state=0))])


### Testing

In [20]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.5429167644854836
0.5427848101265823
Adewale Adeniji 0.542850787306033


### What Accuracy Changes Were Used

What was done to try to increase accuracy and/or limit overfitting:
<ul> 
<li>The first step was to change the diffult train_size of the train_test_split to 0.7 so as to have more data for testing.
<li>Secondly, the max_depth of the DecisionTreeClassifier was set to 5 as this was found to produce the best result for the test data.
</ul>